In [2]:
import math
import os
import cv2
import shutil
import numpy as np
import pandas as pd

from osgeo import gdal, osr
from tqdm import tqdm

In [3]:
SOURCE = '02. Hasil Scan Rename'
TEMP_ROI = 'temp_file/ROI'
TEMP_CROP = 'temp_file/Crop'
COORD = 'sls_st_coordinates.csv'
RESULT = 'georeferenced_maps_export'

In [22]:
def get_loc(input_path, output_path):
  x = 0
  y = 50
  
  # Load the image
  image = cv2.imread(input_path)
  
  height, width = image.shape[:2]
  
  if width > height:
    width = 2650
    height = 2400
    
  else:
    width = 2400
    height = 2650
  
  # Crop the image using the provided coordinates and dimensions
  cropped_image = image[y:y+height, x:x+width]

  # Save the cropped image
  cv2.imwrite(output_path, cropped_image)

In [5]:
def crop_map(input_path, output_path):
  # Load the JPEG file
  img = cv2.imread(input_path)

  # Convert the image to grayscale
  gray = cv2.cvtColor(
    img, 
    cv2.COLOR_BGR2GRAY
  )

  # Apply a Canny edge detection filter to find the edges
  edges = cv2.Canny(gray, 50, 150)

  # Find contours in the image
  contours, hierarchy = cv2.findContours(
    edges, 
    cv2.RETR_EXTERNAL, 
    cv2.CHAIN_APPROX_SIMPLE
  )

  # Find the largest contour
  largest_contour = max(
    contours, 
    key=cv2.contourArea
  )

  # Get the bounding box coordinates
  x, y, w, h = cv2.boundingRect(largest_contour)

  # Extract the image inside the bounding box
  img_cropped = img[y:y+h, x:x+w]

  # Save the cropped image as a new JPEG file
  cv2.imwrite(output_path, img_cropped)

In [6]:
def georeference(input_path, output_path, xmin, xmax, ymin, ymax):
  # Load the input image
  input_image = gdal.Open(input_path)

  # Get the image dimensions
  image_width = input_image.RasterXSize
  image_height = input_image.RasterYSize
  num_bands = input_image.RasterCount

  # Create a spatial reference for the output GeoTIFF
  output_srs = osr.SpatialReference()
  output_srs.ImportFromEPSG(4326)  # Use WGS84 coordinate system

  # Create the output GeoTIFF file
  driver = gdal.GetDriverByName('GTiff')
  output_image = driver.Create(
    output_path, 
    image_width, 
    image_height, 
    num_bands, 
    gdal.GDT_Byte
  )

  # Write the output image data
  for band_index in range(1, num_bands + 1):
      input_band = input_image.GetRasterBand(band_index)
      output_band = output_image.GetRasterBand(band_index)
      output_band.WriteArray(input_band.ReadAsArray())

  # Set the output image geotransform
  geotransform = (
    xmin, 
    (xmax - xmin) / image_width, 
    0, 
    ymax, 
    0, 
    (ymin - ymax) / image_height
  )
  output_image.SetGeoTransform(geotransform)

  # Set the output image projection
  output_image.SetProjection(output_srs.ExportToWkt())

  # Close the input and output images
  input_image = None
  output_image = None

In [7]:
if not os.path.exists(TEMP_CROP):
  os.makedirs(TEMP_CROP)
  
if not os.path.exists(TEMP_ROI):
  os.makedirs(TEMP_ROI)

In [8]:
if not os.path.exists(RESULT):
  os.makedirs(RESULT)

In [12]:
list_peta = os.listdir(SOURCE)
df = pd.read_csv(COORD, dtype={'idsls': str})

In [23]:
for i in tqdm(list_peta):
  name = i.split('_')[0]
  
  # file_path = 'rename/{}_WS.jpg'.format(name) 
  # angel = compute_skew(file_path)
  # dst = deskew(file_path,angel)
  # cv2.imwrite('temp_file/crop/{}_WS.jpg'.format(name), dst)
  
  get_loc(
    '{}/{}_ST2023.jpg'.format(SOURCE, name),
    '{}/{}_ST2023.jpg'.format(TEMP_ROI, name)
  )
  
  crop_map(
    '{}/{}_ST2023.jpg'.format(TEMP_ROI, name),
    '{}/{}_ST2023.jpg'.format(TEMP_CROP, name)
  )
  
  xmin, xmax, ymin, ymax = df.loc[
    df['idsls']==name, 
    ['xmin', 'xmax', 'ymin', 'ymax']
  ].values[0]
  # print(xmin, xmax, ymin, ymax)

  georeference(
    '{}/{}_ST2023.jpg'.format(TEMP_CROP, name), 
    '{}/{}_ST2023.tif'.format(RESULT, name),
    xmin, xmax, ymin, ymax
  )

shutil.rmtree('RESULT')

100%|██████████████████████████████████████████████████████████████████████████████████| 19/19 [00:08<00:00,  2.26it/s]
